# Gen AI 기반 대화형 챗봇 구현 (약 30분 소요)

### Bedrock 클라이언트 설정 및 연결 확인

In [4]:
import json
import boto3
from botocore.config import Config
from pprint import pprint

session = boto3.Session()
region_name = 'us-east-1'

retry_config = Config(
        region_name=region_name,
        retries={
            "max_attempts": 10,
            "mode": "standard",
        },
    )
# Bedrock 클라이언트 설정
bedrock = boto3.client("bedrock", region_name=region_name, config=retry_config)
bedrock_runtime = session.client("bedrock-runtime", region_name=region_name, config=retry_config)

# # 사용 가능한 Foundation 모델 목록 조회
# model_list = bedrock.list_foundation_models()
# result = [(fm["modelName"], fm["modelId"]) for fm in model_list["modelSummaries"] if 'ON_DEMAND' in fm['inferenceTypesSupported']]
# pprint(result)


### 응답의 출력 함수 정의

In [5]:
# 실시간 스트리밍으로 제공되는 Response의 delta 값을 즉시 출력하기 위한 모듈
def bedrock_streamer(response):
    stream = response.get('body')
    answer = ""
    i = 1
    if stream:
        for event in stream:
            chunk = event.get('chunk')
            if  chunk:
                chunk_obj = json.loads(chunk.get('bytes').decode())
                if "delta" in chunk_obj:                    
                    delta = chunk_obj['delta']
                    if "text" in delta:
                        text=delta['text'] 
                        print(text, end="")
                        answer+=str(text)       
                        i+=1
    return answer

### 입력 형식 정의

프롬프트를 모델의 입력 템플릿에 맞춰 저장합니다.

In [6]:
# 프롬프트 기본 구성 정의
prompt_data = "한국 리테일/CPG 산업의 동향에 대해 자세히 알려주세요"

prompt_config = {
    "anthropic_version": "bedrock-2023-05-31",
    "max_tokens": 4096,
    "temperature" : 0,
    "top_k": 350,
    "top_p": 0.999,
    "messages": [
        {
            "role": "user",
            "content": [
                {"type": "text", "text": prompt_data},
            ],
        }
    ],
}

body = json.dumps(prompt_config)

### 모델 호출 - `invoke_model_with_response_stream()`

In [7]:
from IPython.display import clear_output, display, display_markdown, Markdown

modelId = "anthropic.claude-3-sonnet-20240229-v1:0"
accept = "application/json"
contentType = "application/json"

try:

    response = bedrock_runtime.invoke_model_with_response_stream(
        body=body, modelId=modelId, accept=accept, contentType=contentType
    )
    results=bedrock_streamer(response)

except botocore.exceptions.ClientError as error:

    if error.response['Error']['Code'] == 'AccessDeniedException':
           print(f"\x1b[41m{error.response['Error']['Message']}\
                \nTo troubeshoot this issue please refer to the following resources.\
                 \nhttps://docs.aws.amazon.com/IAM/latest/UserGuide/troubleshoot_access-denied.html\
                 \nhttps://docs.aws.amazon.com/bedrock/latest/userguide/security-iam.html\x1b[0m\n")

    else:
        raise error

한국의 리테일/CPG(소비재) 산업은 최근 몇 가지 주요 동향을 보이고 있습니다.

1. 온라인 쇼핑 증가
코로나19 팬데믹으로 인해 비대면 소비가 크게 늘어나면서 온라인 쇼핑 시장이 급성장했습니다. 특히 식료품, 생활용품 등의 온라인 구매가 큰 폭으로 증가했습니다. 이에 따라 기업들은 온라인 판매 채널 강화에 주력하고 있습니다.

2. 편의점 시장 경쟁 심화
편의점 시장이 포화 상태에 이르면서 기존 주요 편의점 브랜드 간 경쟁이 치열해지고 있습니다. 상품 다양화, 배달서비스 강화 등 차별화 전략을 내세우고 있습니다.

3. 프리미엄/HMR 제품 인기
소비자들의 웰빙 트렌드와 더불어 간편하고 고급스러운 제품에 대한 수요가 높아지고 있습니다. 프리미엄 식품, HMR(가정간편식) 제품 등이 인기를 끌고 있습니다.

4. 친환경/지속가능 제품 관심 증가  
환경과 지속가능성에 대한 관심이 높아지면서 친환경 포장, 재활용 원료 사용 등 지속가능한 제품을 선호하는 추세입니다.

5. 기술 활용 강화
인공지능, 빅데이터 등 신기술을 활용해 소비자 니즈 파악, 상품기획, 물류 효율화 등을 도모하는 기업들이 늘고 있습니다.

이처럼 한국 리테일/CPG 시장은 디지털 전환, 프리미엄화, 친환경 트렌드 등 다양한 변화를 겪고 있으며, 기업들은 이에 발맞춰 혁신을 모색하고 있습니다.

### 챗봇에 기본 대화 기능 추가
위에서 우리는 Gen AI 모델과 상호작용 하기 위한 기본 대화기능을 실습했습니다.

아래에서는 이 대화 기능을 Streamlit 애플리케이션의 챗봇에 반영합니다.

In [9]:
%%writefile basic.py
import boto3
import json
import streamlit as st
from botocore.config import Config
from langchain.callbacks.base import BaseCallbackHandler

###### Bedrock 클라이언트 생성 부분 ######
session = boto3.Session()
bedrock_runtime = session.client(
    service_name="bedrock-runtime",
    config=Config(
        region_name='us-east-1',
        retries={"max_attempts": 10, "mode": "standard"}
    )
)

###### 스트리밍 응답 처리 ######
class StreamHandler(BaseCallbackHandler):
    def __init__(self, placeholder):
        super().__init__()
        self.placeholder = placeholder
        self.accumulated_text = ""

    def on_llm_new_token(self, token: str, **kwargs):
        self.accumulated_text += token
        self.placeholder.text(self.accumulated_text)

###### 메시지 처리 ######
def search_basic(prompt, chat_box):    
    prompt_config = {
        "anthropic_version": "bedrock-2023-05-31",
        "max_tokens": 4096,
        "temperature": 0,
        "top_k": 350,
        "top_p": 0.999,
        "messages": [{"role": "user", "content": prompt}],
    }
    body = json.dumps(prompt_config)
    stream_handler = StreamHandler(chat_box)
    
    try:
        ###### Bedrock 호출 부분 ######
        response = bedrock_runtime.invoke_model_with_response_stream(
            body=body, modelId="anthropic.claude-3-sonnet-20240229-v1:0",
            accept="application/json", contentType="application/json"
        )
        for event in response.get('body'):
            chunk = event.get('chunk')
            if chunk:
                chunk_obj = json.loads(chunk.get('bytes').decode())
                delta = chunk_obj.get('delta')
                if delta and "text" in delta:
                    text = delta['text']
                    stream_handler.on_llm_new_token(text)
    except Exception as error:
        st.error(f"Error: {str(error)}")


Overwriting basic.py


In [10]:
%%writefile demo-app.py
from basic import search_basic  # 수정된 부분
import streamlit as st

uploaded_file = st.file_uploader("검색에 사용할 파일을 업로드하세요", type=["pdf", "docx", "pptx"], accept_multiple_files=False)
prompt = st.text_input("프롬프트를 입력하세요.")
search_type = st.radio("Search Type", ["Basic", "Basic-RAG", "Hybrid-RAG", "Advanced-RAG"])

chat_box = st.empty()

def search_documents(search_type: str, prompt: str, file, chat_box):
    if search_type == "Basic":
        search_basic(prompt, chat_box)   # 수정된 부분
    elif search_type == "Basic-RAG":
        st.write(f"문서 기본 검색: {prompt}")
    elif search_type == "Hybrid-RAG":
        st.write(f"하이브리드 검색: {prompt}")
    elif search_type == "Advanced-RAG":
        st.write(f"고급 검색: {prompt}")

if st.button("검색"):
    search_documents(search_type, prompt, uploaded_file, chat_box)

Overwriting demo-app.py


### 같은 질문을 Streamlit 애플리케이션에 입력
"한국 리테일/CPG 산업의 동향에 대해 알려주세요"

<img src="./image/basic-1.png" width="800">

In [66]:
%%writefile basic.py
import boto3
import json
import streamlit as st
from botocore.config import Config
from langchain.callbacks.base import BaseCallbackHandler

###### Bedrock 클라이언트 생성 부분 ######
session = boto3.Session()
bedrock_runtime = session.client(
    service_name="bedrock-runtime",
    config=Config(
        region_name='us-east-1',
        retries={"max_attempts": 10, "mode": "standard"}
    )
)

###### 스트리밍 응답 처리 ######
class StreamHandler(BaseCallbackHandler):
    def __init__(self, placeholder):
        super().__init__()
        self.placeholder = placeholder
        self.accumulated_text = ""

    def on_llm_new_token(self, token: str, **kwargs):
        self.accumulated_text += token
        self.placeholder.text(self.accumulated_text)

###### 메시지 처리 ######
def search_basic(prompt, chat_box):    
    prompt_config = {
        "anthropic_version": "bedrock-2023-05-31",
        "max_tokens": 4096,
        "temperature": 0,
        "top_k": 350,
        "top_p": 0.999,
        "messages": [{"role": "user", "content": prompt}],
    }
    body = json.dumps(prompt_config)
    stream_handler = StreamHandler(chat_box)
    
    try:
        ###### Bedrock 호출 부분 ######
        response = bedrock_runtime.invoke_model_with_response_stream(
            body=body, modelId="anthropic.claude-3-sonnet-20240229-v1:0",
            accept="application/json", contentType="application/json"
        )
        for event in response.get('body'):
            chunk = event.get('chunk')
            if chunk:
                chunk_obj = json.loads(chunk.get('bytes').decode())
                delta = chunk_obj.get('delta')
                if delta and "text" in delta:
                    text = delta['text']
                    stream_handler.on_llm_new_token(text)
    except Exception as error:
        st.error(f"Error: {str(error)}")


Overwriting basic.py


하지만 현재 애플리케이션에서는 대화의 문맥이 유지되지 않습니다.

예를 들어, "한 문장으로 요약해주세요"라고 요청했을 때, 챗봇은 문맥에 맞지않는 답변을 제공할 것입니다.

#### 아래에서는 채팅 애플리케이션 설계를 위한 Langchain 활용방법을 알아봅니다

### Langchain 프레임워크 통합

In [11]:
# Langchain 라이브러리 불러오기
from langchain_community.chat_models import BedrockChat
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain.prompts import PromptTemplate
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory

In [12]:
llmchat = BedrockChat(
    model_id=modelId,
    streaming=True,
    region_name=region_name,
    callbacks=[StreamingStdOutCallbackHandler()],
    model_kwargs={
        "anthropic_version": "bedrock-2023-05-31",
        "max_tokens": 4096,
        "temperature" : 0,
        "top_k": 350,
        "top_p": 0.999
    }
)

In [13]:
prompt_template = """
    Human: You're an advanced AI assistant. Please provide a short answer about my query.
    <context>
    {history}
    </context>
    query - {input}
    
    Assistant:
"""

PROMPT = PromptTemplate(
    template=prompt_template,
    input_variables=['history', 'input']
)

conversation = ConversationChain(
    llm=llmchat, 
    verbose=False, 
    memory=ConversationBufferMemory(),
    prompt=PROMPT
)

### `BedrockChat()` 활용
Bedrock 세션과 직접 통신할 때는 low level API인 `invoke_model_with_response_stream()`을 사용했습니다.

반면, Langchain에서는 `BedrockChat()` Chat Model이 제공하는 high level API인 `ConversationChain`의 `predict()` 메소드를 사용합니다.

*위에 정의한 프롬프트 템플릿(prompt_template)에 따라 답변의 양상은 크게 달라집니다.

In [14]:
conversation.predict(input="한국 리테일/CPG 산업의 동향에 대해 자세히 알려주세요")

한국의 리테일/CPG 산업은 최근 몇 가지 주요 동향을 보이고 있습니다.

1. 온라인 쇼핑 성장: 코로나19 팬데믹으로 인해 비대면 소비가 증가하면서 온라인 쇼핑 시장이 크게 성장했습니다. 

2. 옴니채널 전략: 오프라인 매장과 온라인 채널을 연계한 옴니채널 마케팅이 중요해지고 있습니다.

3. 친환경/웰니스 트렌드: 환경과 건강에 대한 관심 증가로 친환경, 웰니스 제품 수요가 늘고 있습니다.  

4. 고객 경험 중시: 차별화된 고객 경험을 제공하기 위해 매장 환경, 서비스 개선에 주력하고 있습니다.

5. 기술 활용: AI, 빅데이터 등 신기술을 활용한 마케팅, 운영 효율화가 이루어지고 있습니다.

이와 같은 주요 동향을 반영하여 리테일/CPG 기업들이 전략을 수립하고 있습니다.

'한국의 리테일/CPG 산업은 최근 몇 가지 주요 동향을 보이고 있습니다.\n\n1. 온라인 쇼핑 성장: 코로나19 팬데믹으로 인해 비대면 소비가 증가하면서 온라인 쇼핑 시장이 크게 성장했습니다. \n\n2. 옴니채널 전략: 오프라인 매장과 온라인 채널을 연계한 옴니채널 마케팅이 중요해지고 있습니다.\n\n3. 친환경/웰니스 트렌드: 환경과 건강에 대한 관심 증가로 친환경, 웰니스 제품 수요가 늘고 있습니다.  \n\n4. 고객 경험 중시: 차별화된 고객 경험을 제공하기 위해 매장 환경, 서비스 개선에 주력하고 있습니다.\n\n5. 기술 활용: AI, 빅데이터 등 신기술을 활용한 마케팅, 운영 효율화가 이루어지고 있습니다.\n\n이와 같은 주요 동향을 반영하여 리테일/CPG 기업들이 전략을 수립하고 있습니다.'

#### 이제 대화의 컨텍스트가 유지되고 있는지 확인해봅니다

In [15]:
conversation.predict(input="한 문장으로 요약해주세요")

한국 리테일/CPG 산업은 온라인 쇼핑 성장, 옴니채널 전략, 친환경/웰니스 트렌드, 고객 경험 중시, 신기술 활용 등의 주요 동향을 보이며 변화하고 있습니다.

'한국 리테일/CPG 산업은 온라인 쇼핑 성장, 옴니채널 전략, 친환경/웰니스 트렌드, 고객 경험 중시, 신기술 활용 등의 주요 동향을 보이며 변화하고 있습니다.'

#### 챗봇을 완성하기 위해 우리는 아래 작업을 진행했습니다.
1. Bedrock 기반의 Chat Model 인스턴스 `BedrockChat()`를 생성합니다.
2. 대화를 유지하기 위한 `ConversationChain` 생성합니다. `ConversationBufferMemory`에는 대화 내용이 저장됩니다.
3. 위 ConversationChain에 predict() 호출로 메시지를 전달하여 스트리밍 응답을 받습니다.

### Streamlit 애플리케이션 업데이트

이제 위에서 작업했던 내용을 Streamlit 애플리케이션 코드에 반영해봅니다

In [16]:
%%writefile basic.py
import streamlit as st
from langchain_community.chat_models import BedrockChat
from langchain.callbacks.base import BaseCallbackHandler
from langchain.prompts import PromptTemplate
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory

region_name = 'us-east-1'
modelId = "anthropic.claude-3-sonnet-20240229-v1:0"

###### 스트리밍 응답 처리 ######
class StreamHandler(BaseCallbackHandler):
    def __init__(self, placeholder):
        super().__init__()
        self.placeholder = placeholder
        self.accumulated_text = ""

    def reset_accumulated_text(self):
        self.accumulated_text = ""
        self.placeholder.text(self.accumulated_text)

    def on_llm_new_token(self, token: str, **kwargs):
        self.accumulated_text += token
        self.placeholder.text(self.accumulated_text)

def get_conversation(chat_box):
    stream_handler = StreamHandler(chat_box)
    
    llmchat = BedrockChat(
        model_id=modelId,
        streaming=True,
        region_name=region_name,
        callbacks=[stream_handler], 
        model_kwargs={
            "anthropic_version": "bedrock-2023-05-31",
            "max_tokens": 4096,
            "temperature": 0,
            "top_k": 350,
            "top_p": 0.999
        }
    )

    prompt_template = """
    Human: You're an advanced AI assistant. Please provide a short answer about my query.
    <context>
    {history}
    </context>
    query - {input}
    
    Assistant:
    """

    PROMPT = PromptTemplate(
        template=prompt_template,
        input_variables=['history', 'input']
    )

    conversation = ConversationChain(
        llm=llmchat, 
        verbose=False, 
        memory=ConversationBufferMemory(human_prefix="Human", ai_prefix="Assistant"),
        prompt=PROMPT
    )

    return conversation, stream_handler



Overwriting basic.py


In [17]:
%%writefile demo-app.py
from basic import get_conversation
import streamlit as st

uploaded_file = st.file_uploader("검색에 사용할 파일을 업로드하세요", type=["pdf", "docx", "pptx"], accept_multiple_files=False)
prompt = st.text_input("프롬프트를 입력하세요.")
search_type = st.radio("Search Type", ["Basic", "Basic-RAG", "Hybrid-RAG", "Advanced-RAG"])

chat_box = st.empty()

if 'conversation' not in st.session_state or 'stream_handler' not in st.session_state:  # 수정된 부분 (대화 내용 유지를 위해 streamlit 세션 관리가 필요합니다)
    st.session_state.conversation, st.session_state.stream_handler = get_conversation(chat_box)

def search_documents(search_type: str, prompt: str):
    if search_type == "Basic":
        st.session_state.stream_handler.reset_accumulated_text()
        st.session_state.conversation.predict(input=prompt)
    elif search_type == "Basic-RAG":
        st.write(f"문서 기본 검색: {prompt}")
    elif search_type == "Hybrid-RAG":
        st.write(f"하이브리드 검색: {prompt}")
    elif search_type == "Advanced-RAG":
        st.write(f"고급 검색: {prompt}")

if st.button("검색"):
    search_documents(search_type, prompt)


Overwriting demo-app.py


#### 이제 챗봇이 과거 대화내용을 잘 기억하고 있습니다.

<img src="./image/basic-2.png" width="800">

#### 이제 생성형 AI 기반 대화기능을 갖춘 챗봇이 완성됐습니다. 

그런데 우리는 이 챗봇을 특정 사내 지식기반 또는 최신 데이터에 특화시키고 싶습니다.

#### 챗봇에 검색증강생성(RAG) 기능을 추가하기 위해 `2-basic-rag.ipynb` 노트북으로 이동합니다.